In [2]:
import keras
from keras.datasets import imdb
import numpy as np
import nltk
from nltk.stem.lancaster import LancasterStemmer
import json
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import csv
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [31]:
imdb_path = "/media/roodram/72F8C754F8C7156F/ACM/Project/DataSets/imdb.npz"
(x1, y1), (x2, y2)= imdb.load_data(path = imdb_path, maxlen=2000, num_words=50000)

In [32]:
print(type(x1[0]))
maxlen=2000
num_words=50000

<class 'list'>


In [33]:
x = np.concatenate((x1,x2), axis=0)
y = np.concatenate((y1,y2), axis=0)
print(x.shape)
print(y.shape)


(49996,)
(49996,)


In [10]:
num_words=0
maxlen=0
for i in range(x.shape[0]):
    #print(i)
    l = len(x[i])
    maxlen = max(maxlen, l)
    num_words = max(num_words, max(x[i]))
    
print(num_words)
print(maxlen)

88586
2494


In [34]:
x = sequence.pad_sequences(x, maxlen=maxlen+1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
print(x_train.shape)
print(y_train.shape)
print(len(x[0]))

(39996, 2001)
(39996,)
2001


In [14]:
from keras.models import Model
from keras.layers import Dense, Embedding, RNN, Input, Dropout, LSTM, Activation
from keras.utils import to_categorical
from keras.optimizers import Adam

In [26]:
def RNN_Model_Keras(maxlen, num_words):
    inputs = Input((maxlen,), dtype=np.float32)
    X = Embedding(num_words, 64, input_length=maxlen)(inputs)
    X = LSTM(128)(X)
    X = Dense(1, activation='sigmoid')(X)
    model = Model(inputs=inputs, outputs=X)
    return model

In [36]:
model = RNN_Model_Keras(maxlen+1, num_words+1)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 2001)              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 2001, 64)          3200064   
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 3,299,009
Trainable params: 3,299,009
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs = 3, batch_size = 5, shuffle=True)

Epoch 1/3
21940/39996 [===============>..............] - ETA: 1:18:11 - loss: 0.4404 - acc: 0.7972

In [ ]:
loss, acc = model.evaluate(x_test, y_test)
print(loss)
print(acc)